# Eternal Card Extractor

## Section A: Make sure you have the required python 3 modules
This script was built on Python 3.x and uses pyautogui and opencv 3.4.2. I think the rest of the default Python 3 modules are safe. If the cells below throw errors, you may have problems.

In [1]:
import sys
assert sys.version[0]=='3'

In [2]:
import cv2
assert cv2.__version__=='3.4.2'
    # cv2 versions above 3.4.2 do not seem to have xfeatures2d.SIFT methods directly available.
    # When in doubt, force the installation of version 3.4.2.16. I used
    #   pip3 install --user opencv-python==3.4.2.16
    #   pip3 install --user opencv-contrib-python==3.4.2.16
    # Both opencv-python and open-contrib-python seemed to be needed for SIFT to work.

In [3]:
import pyautogui
import pickle
import gzip

## Section B: Grab screenshots of your card collection
For this section, get Eternal running (maximized is easier) and head to the first screen of your card collection. Afterwards Alt-Tab back to this Jupyter notebook, make sure it is unmaximized and in a position that leaves Eternal visible on the top left corner (so that Python can simulate a click on the Eternal client and bring it to the foreground to take the screenshots), and run the cells below.

(DISCLAIMER: I did not actually run the Section B code from this Jupyter notebook - I ran it from the command line using a script because I do not have Jupyter on the machine where I have Eternal installed. The script I did use is also available on the github repo. Be careful in any case: the script for reasons unknown generates two screenshots of the first page of the collection, which is ok but meeses up the number of cards owned of those cards. If you end up with duplicate screenshots, clean up!)

In [8]:
import pyautogui
import PIL
from PIL import ImageChops
import numpy as np
import time

In [5]:
screenshots_folder='./Screenshots/'
    # Where to save screenshots
max_screenshots=150
    # Stop after so many screenshots if the end condition is not met
avg_diff_cutoff=5
    # Screenshot average pixel value to consider two screenshots identical
same_img_limit=3
    # Number of identical sequential screenshots to determine the end of the collection has been reached
eternal_x,eternal_y=10,10
    # Position to click on in order to bring Eternal to the foreground
next_x,next_y=0.855*pyautogui.size()[0],0.525*pyautogui.size()[1]
    # Position to click to move to the next collection page

In [6]:
remove_new_aura=False
    # Flag signaling whether the shinyness around new cards should be removed
    # before taking the screenshots
sep_x,sep_y=int(0.1218*pyautogui.size()[0]),int(0.3852*pyautogui.size()[1])
    # Represents the distances between equivalent positions of cards on the screenshot,
    # i.e., 'top left to top left' or 'center to center', or whatever
crd_x,crd_y=int(0.1730*pyautogui.size()[0]),int(0.2120*pyautogui.size()[1])
    # Represents the top left corner of the feature extraction box of the first card
pos_lst=[ (crd_x+n_x*sep_x,crd_y+n_y*sep_y)
          for n_y in range(2) for n_x in range(6) ]
    # Represent a position in each of the 12 cards so that the script can remove the 'NEW' aura

In [9]:
pyautogui.click(eternal_x,eternal_y)

cnt_ss=0
cnt_page=0
cnt_same=0
old_screenshot=PIL.Image.new('RGB',pyautogui.size())
while (cnt_ss<max_screenshots) and (cnt_same<same_img_limit):
    cnt_ss=cnt_ss+1
    if remove_new_aura:
        time.sleep(0.1)
        for card_x,card_y in pos_lst:
            pyautogui.moveTo(card_x,card_y)
        pyautogui.moveTo(next_x,next_y)
    new_screenshot=pyautogui.screenshot()
    if np.asarray(ImageChops.difference(old_screenshot,new_screenshot)).mean()>avg_diff_cutoff:
        cnt_same=0
        cnt_page=cnt_page+1
        new_screenshot.save(screenshots_folder+'ss{:03d}.png'.format(cnt_page))
        old_screenshot=new_screenshot
    else:
        cnt_same=cnt_same+1
    pyautogui.click(next_x,next_y)

## Section C: Identify card collection

### Section C.1: Load card artwork features

The library is based on the one in Eternal Warcry - thanks for allowing me to use it for this project!

The pickle module is very nice but can lead to incompatibilities. If it cannot unpickle the library, let me know so that I can think of an alternative way of serializing it.

In [10]:
import os
import pickle
import gzip

In [11]:
library_local='./Libraries/'
    # Location of the pickled gzipped libraries with the artwork descriptors already calculated
    # that I prepared so that you don't have to.
library_root='card_library_set_'
    # Expected name of the library files - in case any other files rnd up in the same folder.

In [12]:
library={}
for set_library in os.listdir(library_local):
    if library_root in set_library:
        with gzip.open(library_local+set_library,'rb') as library_file:
            library.update(pickle.load(library_file))

### Section C.2: Process screenshots

In [13]:
import pyautogui
import cv2
import os

In [14]:
img_w,img_h=int(0.0479*pyautogui.size()[0]),int(0.1130*pyautogui.size()[1])
    # Size in pixels of the box used to extract features from.
    # Limited by either the resolution of your screen or that of the card artwork.
nfeatures=50
    # Number of features to extract from each box. 50 seems to work well.

    # This area has some upside potential by the way - identifying less features based
    # on lower resolution boxes would speed up the process and produce a smaller
    # card library file. To be explored.
sift=cv2.xfeatures2d.SIFT_create(nfeatures=nfeatures)

The functions below seem fairly common for this specific purpose. My understanding of them is incomplete, but the first one seems to return the good matching features between two descriptor sets (and will be used once between the target and each of the possible option images), and the second one seems to compute how may of the total matching features that were considered good came from each classification option (and will be used once, after all of the options have been evaluated).

The option that generated the highest number of good matching features is the most likely one.

In [15]:
def knn_match(des1, des2, nn_ratio=0.7):
    index_params = dict(algorithm = 0, trees = 5)
    search_params = dict(checks = 50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    good = []
    for m, n in matches:
        if m.distance < nn_ratio * n.distance:
            good.append(m)
    return good

def knn_clasif(good_matches):
    logprobs=[]
    best_template, highest_logprob = None, 0.0
    sum_good_matches = sum([len(gm) for gm in good_matches])
    for i, gm in enumerate(good_matches):
        logprob = len(gm)/sum_good_matches
        logprobs.append(logprob)
        if logprob > highest_logprob:
            highest_logprob = logprob
            best_template = i
    return best_template,logprobs

The parameters below define the places in the screenshots where we will extract image boxes and number owned diamonds from. Plenty of options would be available; I went for estimating:
- The distance between cards (sep_x,sep_y), which when aplied to the position of the top left corner of the first image box (crd_x,crd_y) generates the boxes that will be used to extract owned card features (pos_lst)
- The distance between the cards (the same sep_x,sep_y) and the horizontal distance between the ownership diamonds (sep_cnt), which when applied to the position of the center of the second diamond (the first one is a given - you always have at least one card of those present in the screenshots) of the top left card (cnt_x,cnt_y) generates the points to check if an ownership diamond is present (cnt_lst). This test is done against a cutoff of 100, but the actual values measured on the screenshots were 330 on yes-diamond and 15 on no-diamond

There is another notebook in the repo to help finetune these parameters.

In [16]:
screenshots_folder='./Screenshots/'
collection_file='./my_collection.txt'

sep_x,sep_y=int(0.1218*pyautogui.size()[0]),int(0.3852*pyautogui.size()[1])
    # Represents the distances between equivalent positions of cards on the screenshot,
    # i.e., 'top left to top left' or 'center to center', or whatever
crd_x,crd_y=int(0.1730*pyautogui.size()[0]),int(0.2120*pyautogui.size()[1])
    # Represents the top left corner of the feature extraction box of the first card
pos_lst=[ (crd_x+n_x*sep_x,crd_y+n_y*sep_y,img_w,img_h)
          for n_y in range(2) for n_x in range(6)]
    # Represent the list of feature extraction boxes of the 12 cards

sep_cnt=int(0.013*pyautogui.size()[0])
    # Represents the horizontal distance between equivalent positions of diamods on the screenshot
    # i.e., 'top left to top left' or 'center to center', or whatever
cnt_x,cnt_y=int(0.1917*pyautogui.size()[0]),int(0.1463*pyautogui.size()[1])
    # Represents the center of the second ownership diamond of the first card
cnt_lst=[ [(cnt_x+n_x*sep_x+n_cnt*sep_cnt,cnt_y+n_y*sep_y) for n_cnt in range(3)]
          for n_y in range(2) for n_x in range(6)]
    # Represents the center of the second, third, and fourth diamonds of the 12 cards

owned_threshold=100
    # Threshold to determine whether at the point there is a yes-diamond or no-diamond.
    # The actual numbers are 330 for yes-diamond and 15 for no-diamond

The script extracts each card in each screenshot of the collection, compares its features with those of all the cards in the library, and finds the best match.

It can be made much faster by identifying additional features from the cards, such as power requirements profile, cost, or attack; these can then be used to narrow the search for the target card, which would also allow to reduce the size of the feature extraction boxes and the number of features being extracted from each box.

I also tried text extraction, but tesseract did not work well on the card names, at least not on my low resolution screenshots. That however does not mean it cannot be used to speed up the identification process, as long as the extracted gibberish is consistent.

To be explored.

In [17]:
%%time

list_cards=list(library)
for card in library:
    library[card]['owned']=0
errors=[]
tot=len(os.listdir(screenshots_folder))
cnt=0
for ss_file in sorted(os.listdir(screenshots_folder)):
    cnt=cnt+1
    print('[{:3d}/{:3d}] Processing file {}...'.format(cnt,tot,ss_file))
    if ss_file[-3:]=='png':
        screenshot=cv2.imread(screenshots_folder+ss_file)
        for i in range(12):
            l,t,w,h=pos_lst[i]
            target=cv2.resize(screenshot[t:t+h,l:l+w],(img_w,img_h))
            try:
                _,descriptors=sift.detectAndCompute(target,None)
                list_good_matches=[]
                for card in library:
                    list_good_matches.append(knn_match(library[card]['descriptors'],descriptors))
                best_match,probs=knn_clasif(list_good_matches)
                num_owned=1+sum([ screenshot[c[1],c[0],:].sum()>owned_threshold for c in cnt_lst[i]])
                print(12*' '+'- {} '.format(num_owned)+list_cards[best_match])
                library[list_cards[best_match]]['owned']=min(4,library[list_cards[best_match]]['owned']+num_owned)
            except:
                errors.append('[ERROR] Problem identifying card {:2d} in screenshot {}!'.format(i+1,ss_file))
                print(errors[-1])
                pass

[  1/117] Processing file ss001.png...
            - 4 Seek Power
            - 1 Fire Sigil
            - 4 Granite Monument
            - 1 Granite Monument
            - 4 Granite Waystone
            - 4 Shugo Standard
            - 4 Helpful Doorbot
            - 4 Melt Down
            - 1 Trail Stories
            - 4 Bore
            - 1 Bottoms Up
            - 4 Cautious Traveler
[  2/117] Processing file ss002.png...
            - 4 Drifter
            - 4 Firemane Cub
            - 4 Forge Wolf
            - 4 Grenadin Drone
            - 4 Heavy Axe
            - 4 Hidden Shiv
            - 1 Hidden Shiv
            - 4 Iceberg Frontrunner
            - 1 Kaleb's Intervention
            - 4 Light 'em Up
            - 2 Mindfire
            - 4 On the Hunt
[  3/117] Processing file ss003.png...
            - 4 Oni Dragonsmith
            - 4 Oni Ronin
            - 4 Pummel
            - 4 Pyroknight
            - 4 Rambot
            - 4 Research Assistant
            - 4

            - 4 Silence
            - 4 Slow
            - 1 Slow
            - 4 Staff of Speed
            - 1 Staff of Speed
            - 4 Training Ground
            - 4 Water of Life
            - 4 Accelerate
            - 4 Bold Adventurer
            - 2 Copper Conduit
[ 22/117] Processing file ss022.png...
            - 2 Cryptic Etchings
            - 1 Cryptic Etchings
            - 4 Disjunction
            - 1 Dissociate
            - 1 Ephemeral Wisp
            - 3 Evelina, Valley Searcher
            - 4 Find the Way
            - 2 Forgotten Find
            - 4 Friendly Wisp
            - 3 Illumination Wisp
            - 2 Last Rites
            - 4 Learned Herbalist
[ 23/117] Processing file ss023.png...
            - 4 Living Example
            - 3 Living Offering
            - 1 Locust
            - 3 Nocturnal Creeper
            - 4 Oasis Seeker
            - 4 Packbeast
            - 4 Porcelain Mask
            - 4 Power Stone
            - 4 Refresh
      

            - 4 Inspire
            - 3 Kosul Diplomat
[ 41/117] Processing file ss041.png...
            - 4 Master-at-Arms
            - 4 Minotaur Grunt
            - 4 Minotaur Grunt
            - 2 Paladin Oathbook
            - 4 Protect
            - 1 Rampart Protector
            - 4 Rebuke
            - 4 Reinforce
            - 2 Reinvigorate
            - 1 Rilgon's Disciple
            - 4 Rolant's Favor
            - 1 Rolant's Favor
[ 42/117] Processing file ss042.png...
            - 4 Soaring Guard
            - 4 Sparring Partner
            - 4 Stalwart Silverwing
            - 4 Strength of Many
            - 4 Talon of Nostrix
            - 4 Tinker
            - 1 Tinker
            - 4 Tinker Overseer
            - 4 Tranquil Scholar
            - 4 Vanquish
            - 1 Wanted Poster
            - 1 Workshop Tinker
[ 43/117] Processing file ss043.png...
            - 1 Workshop Tinker
            - 3 Auric Herdward
            - 4 Auric Sentry
            - 4

            - 2 Mating Call
            - 2 Parry
            - 2 Portent Reader
            - 2 Read the Stars
            - 4 Reinforced Towershield
            - 4 Scaly Gruan
            - 1 Scaly Gruan
            - 4 Second Sight
[ 61/117] Processing file ss061.png...
            - 4 Snowfort Trumpeter
            - 4 Static Bolt
            - 1 Storm Talisman
            - 3 Strategize
            - 4 Talon Dive
            - 1 Talon Dive
            - 4 Tend the Flock
            - 3 Torrential Downpour
            - 4 Trailblaze
            - 2 Twilight Hermit
            - 4 Unseal
            - 4 Violent Gust
[ 62/117] Processing file ss062.png...
            - 4 Wardwielder
            - 1 Whispering Wind
            - 4 Wild Cloudsnake
            - 2 Wind Cloak
            - 4 Winter's Grasp
            - 4 Yeti Snowchucker
            - 4 Yeti Snowslinger
            - 4 Yeti Windflyer
            - 4 Acquisitive Crow
            - 4 Advance Scout
            - 4 Cloudsn

            - 4 Sporefolk
            - 4 Territorial Elf
            - 4 Threaten
            - 1 Threaten
            - 4 Unseen Agent
            - 4 Vampire Bat
[ 81/117] Processing file ss081.png...
            - 4 Vara's Favor
            - 1 Vara's Favor
            - 4 Venomfang Dagger
            - 4 Amethyst Acolyte
            - 4 Amethyst Ring
            - 4 Auric Interrogator
            - 4 Banished Umbren
            - 4 Beastcaller's Amulet
            - 4 Blackguard Sidearm
            - 1 Cabal Countess
            - 3 Cat Burglar
            - 2 Coronal Umbren
[ 82/117] Processing file ss082.png...
            - 1 Coronal Umbren
            - 4 Daggerclaw Howler
            - 4 Desperado
            - 4 Devouring Shadow
            - 4 Direwood Beastcaller
            - 4 Duskcaller
            - 4 Execute
            - 4 Extract
            - 4 Flickerling
            - 4 Hair-Trigger Pistol
            - 1 Hair-Trigger Pistol
            - 4 Lethrai Memory-Keeper


            - 2 Black Iron Manacles
            - 1 Clan Huntcaller
            - 1 Stonescar Pickaxe
            - 3 Jekk's Choice
            - 1 Runic Revolver
            - 4 Warband Chieftain
            - 1 Failed Reflection
            - 1 Infernal Tyrant
            - 4 Smuggler's Stash
[101/117] Processing file ss101.png...
            - 4 Combrei Banner
            - 4 Seat of Progress
            - 3 Alessi, Combrei Archmage
            - 4 Awakened Student
            - 4 Desert Marshal
            - 4 Jaril, Amaran Ghostblade
            - 4 Safe Return
            - 1 Safe Return
            - 4 Combrei Emissary
            - 4 Combrei Healer
            - 1 Combrei Healer
            - 1 Shush
[102/117] Processing file ss102.png...
            - 1 Stand Together
            - 4 Vodakhan's Staff
            - 4 Copperhall Elite
            - 1 Enlightened Stranger
            - 4 Karmic Guardian
            - 1 Penitent Bull
            - 1 Stronghold's Visage
           

In [18]:
my_collection=[]
for card in library:
    if library[card]['owned']>0:
        my_collection.append('{} {} (Set{} #{})'.format(library[card]['owned'],
                                                        card,
                                                        library[card]['set'],
                                                        library[card]['id']))
with open(collection_file,'w') as f:
    f.writelines([card+'\n' for card in my_collection])

Errors are to be expected at the last few cards, where there is actually no card on the screenshot.

In [19]:
errors

['[ERROR] Problem identifying card 10 in screenshot ss117.png!',
 '[ERROR] Problem identifying card 11 in screenshot ss117.png!',
 '[ERROR] Problem identifying card 12 in screenshot ss117.png!']